In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import sys
# import os

# # Lấy đường dẫn thư mục gốc của dự án
# project_root = project_root = '/content/drive/MyDrive/NLP/toxic_comment_vietnamese'

# # Thêm đường dẫn này vào sys.path
# if project_root not in sys.path:
#     sys.path.append(project_root)

In [ ]:
# ! pip -q install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 16.2 MB/s eta 0:00:00


In [ ]:
# from src.data_utils import preprocess_text

# import pandas as pd
# import time
# import re

# from google.colab import userdata
# from tqdm.notebook import tqdm

# **Cleaning data**

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/NLP/toxic_comment_vietnamese/data/raw/youtube_comments.csv')

# df.dropna(subset=['comment'], inplace=True)

# df['preprocessed_text'] = df['comment'].apply(preprocess_text)

# df.to_csv('/content/drive/MyDrive/NLP/toxic_comment_vietnamese/data/processed/comments_preprocessed.csv', index=False)
# print("Đã lưu dữ liệu đã tiền xử lý vào: \n /content/drive/MyDrive/NLP/toxic_comment_vietnamese/data/processed/comments_preprocessed.csv'")

In [ ]:
# print(f"The comments_preprocessed.csv file has {df.shape[0]} samples (rows).")

# **Auto-labeling**

## Fine-tuning PhoBERT on UIT-ViCTSD

In [ ]:
# !pip install -q torch pandas numpy scikit-learn "transformers[sentencepiece]" datasets accelerate

In [ ]:
# !pip install -q --upgrade transformers

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import os
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    DataCollatorWithPadding
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Chọn thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Load tokenizer và model
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: 'non-toxic', 1: 'toxic'},
    label2id={'non-toxic': 0, 'toxic': 1}
)
model.to(device)

# Load dataset
dataset = load_dataset("tarudesu/ViCTSD")

# Tính class weights
train_labels = dataset['train']['Toxicity']
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"✅ Đã tính toán Class Weights: {class_weights}")

# Custom Trainer với compute_loss chấp nhận **kwargs
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fct = nn.CrossEntropyLoss(weight=class_weights_tensor)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Nhận cả các tham số bổ sung như num_items_in_batch qua **kwargs
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.loss_fct(
            logits.view(-1, self.model.config.num_labels),
            labels.view(-1)
        )
        if return_outputs:
            return loss, outputs
        return loss

# Hàm để gán nhãn nhị phân
def create_binary_label(example):
    example['label'] = int(example['Toxicity'] == 1)
    return example

# Số lõi CPU để xử lý map song song
num_cores = os.cpu_count()
print(f"Sử dụng {num_cores} lõi CPU để xử lý dữ liệu.")

# Áp map với num_proc
labeled_dataset = dataset.map(create_binary_label, num_proc=num_cores)
processed_dataset = labeled_dataset.rename_column("Comment", "text")

# Hàm tokenize
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=256)

tokenized_train_dataset = processed_dataset['train'].map(
    tokenize_function, batched=True, num_proc=num_cores
)
tokenized_eval_dataset = processed_dataset['validation'].map(
    tokenize_function, batched=True, num_proc=num_cores
)

# Data collator cho padding động
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Hàm compute_metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

# TrainingArguments — dùng eval_strategy vì bạn dùng transformers mới
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/toxic_comment_vietnamese/models",
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/NLP/toxic_comment_vietnamese/logs",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,  # bật mixed precision nếu GPU hỗ trợ
)

# Khởi tạo Trainer và train
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    data_collator=data_collator,
)

print("🚀 Bắt đầu huấn luyện với các tối ưu cao cấp...")
trainer.train()
print("🎉 Huấn luyện hoàn tất!")


Sử dụng thiết bị: cuda


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Đã tính toán Class Weights: [0.56080756 4.6113307 ]
Sử dụng 2 lõi CPU để xử lý dữ liệu.


Map (num_proc=2):   0%|          | 0/7000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

...

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

🚀 Bắt đầu huấn luyện với các tối ưu cao cấp...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.578300,0.528898,0.653500,0.715725
2,0.540300,0.606370,0.887500,0.882166
3,0.437500,0.472496,0.776500,0.812390
4,0.308400,1.007466,0.899500,0.893004


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.578300,0.528898,0.653500,0.715725
2,0.540300,0.606370,0.887500,0.882166
3,0.437500,0.472496,0.776500,0.812390
4,0.308400,1.007466,0.899500,0.893004
5,0.194000,1.498652,0.899000,0.891551
6,0.091300,1.479799,0.893000,0.889517


🎉 Huấn luyện hoàn tất!


## Auto Labeling (reference on kaggle)

In [ ]:
# Cài đặt thư viện cần thiết
!pip install transformers -q
!pip install torch -q

In [ ]:
import torch
import pandas as pd
import os, math

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm

In [ ]:
# Đường dẫn tới thư mục chứa checkpoint trên Kaggle
MODEL_PATH = "/kaggle/input/vietnamese-toxic-comments/input_kaggle/checkpoint-1314"
RAW_DATA_PATH = "/kaggle/input/vietnamese-toxic-comments/input_kaggle/data/processed/comments_preprocessed.csv" # Đường dẫn tới thư mục data thô
PROCESSED_DATA_PATH = "/kaggle/working/processed/" # Đường dẫn để lưu data đã xử lý

# Tạo thư mục output nếu chưa có
import os
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

# Tải Tokenizer và Model
print("Đang tải tokenizer và model...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
    print("Tải thành công!")
except Exception as e:
    print(f"Lỗi khi tải model: {e}")
    print("Vui lòng kiểm tra lại đường dẫn và đảm bảo các file trong checkpoint đầy đủ.")

# Chuyển model sang GPU nếu có để tăng tốc
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Sử dụng thiết bị: {device}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 12.5 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

2025-10-10 14:18:54.466560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760105934.640426      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760105934.691596      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Tải thành công!
Sử dụng thiết bị: cuda


In [ ]:
# Bước 1: Ánh xạ giữa ID và nhãn
id2label = {
    0: "non-toxic",
    1: "toxic"
}

# Bước 2: Xây dựng hàm dự đoán cho một batch dữ liệu
# Việc xử lý theo batch (lô) sẽ nhanh hơn rất nhiều so với xử lý từng câu một.
def predict_batch(texts):
    """
    Hàm nhận vào một danh sách các câu (texts) và trả về dự đoán của model.
    """
    # Tokenize văn bản
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")

    # Chuyển inputs sang cùng device với model (GPU hoặc CPU)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Đưa inputs vào model để dự đoán mà không cần tính gradient
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits

    # Lấy ra nhãn có xác suất cao nhất bằng hàm argmax
    predictions = torch.argmax(logits, dim=-1)

    # Trả về kết quả dự đoán (dưới dạng tensor)
    return predictions

# Bước 3: Đọc dữ liệu, gán nhãn và lưu kết quả
try:
    print(f"Đang đọc dữ liệu từ: {RAW_DATA_PATH}")
    df = pd.read_csv(RAW_DATA_PATH)

    TEXT_COLUMN = 'preprocessed_text'
    if TEXT_COLUMN not in df.columns:
        raise ValueError(f"Không tìm thấy cột '{TEXT_COLUMN}' trong file CSV. Vui lòng kiểm tra lại.")

    # Xử lý các giá trị NaN (rỗng) trong cột văn bản để tránh lỗi
    df.dropna(subset=[TEXT_COLUMN], inplace=True)
    df[TEXT_COLUMN] = df[TEXT_COLUMN].astype(str)

    # Thiết lập kích thước batch
    # Nếu bạn gặp lỗi hết bộ nhớ (Out of Memory), hãy giảm số này xuống (ví dụ: 16, 8)
    BATCH_SIZE = 32

    all_predictions = []

    print("Bắt đầu quá trình gán nhãn tự động...")
    # Sử dụng tqdm để theo dõi tiến độ
    for i in tqdm(range(0, len(df), BATCH_SIZE)):
        batch_texts = df[TEXT_COLUMN][i:i+BATCH_SIZE].tolist()
        if not batch_texts:
            continue

        # Lấy dự đoán cho batch hiện tại
        batch_preds = predict_batch(batch_texts)

        # Chuyển kết quả từ tensor trên GPU về list trên CPU và thêm vào danh sách tổng
        all_predictions.extend(batch_preds.cpu().numpy())

    # Thêm cột kết quả dự đoán (dưới dạng ID) vào dataframe
    df['predicted_label_id'] = all_predictions

    # Ánh xạ từ ID sang nhãn dạng text (non-toxic, toxic)
    df['predicted_label'] = df['predicted_label_id'].map(id2label)

    # Lưu dataframe đã được gán nhãn ra file CSV mới
    output_path = os.path.join(PROCESSED_DATA_PATH, "comments_auto_labeled.csv")
    df.to_csv(output_path, index=False, encoding='utf-8-sig') # Dùng utf-8-sig để Excel đọc tiếng Việt không bị lỗi

    print("\nHoàn thành!")
    print(f"Đã lưu kết quả vào file: {output_path}")

    # In ra 5 dòng đầu tiên của kết quả để kiểm tra
    print("\nXem trước 5 kết quả đầu tiên:")
    print(df.head())

    # In ra thống kê số lượng các nhãn được dự đoán
    print("\nThống kê số lượng nhãn:")
    print(df['predicted_label'].value_counts())

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file dữ liệu thô tại '{RAW_DATA_PATH}'.")
    print("Vui lòng kiểm tra lại đường dẫn.")
except Exception as e:
    print(f"Đã có lỗi xảy ra: {e}")

Đang đọc dữ liệu từ: /kaggle/input/vietnamese-toxic-comments/input_kaggle/data/processed/comments_preprocessed.csv
Bắt đầu quá trình gán nhãn tự động...


  0%|          | 0/265 [00:00<?, ?it/s]


Hoàn thành!
Đã lưu kết quả vào file: /kaggle/working/processed/comments_auto_labeled.csv

Xem trước 5 kết quả đầu tiên:
                                             comment  \
0  Giống vụ con bé báo tao cháu tô lâm đây . CA b...   
1                  BIẾT BỐ MÀY LÀ AI KHÔNG .\nĐMCS .   
2  Còn cái này là góp ý. Hy vọng là có đọc cmt. N...   
3                        Bò đỏ lên bình luận cái coi   
4  Duoc dao tao chinh qui,hoc truong cao cap ,duo...   

                                   preprocessed_text  predicted_label_id  \
0  giống vụ con bé báo tao cháu tô lâm đây ca bảo...                   0   
1                       biết bố mày là ai không đmcs                   0   
2  còn cái này là góp ý hy vọng là có đọc cmt nói...                   0   
3                        bò đỏ lên bình luận cái coi                   0   
4  duoc dao tao chinh quihoc truong cao cap duoc ...                   0   

  predicted_label  
0       non-toxic  
1       non-toxic  
2       non-toxic  
3    